In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
products = pd.read_csv('/content/drive/My Drive/ML Specialization/amazon_baby.csv')

In [ ]:
import re
import string

def remove_punctuation(text):

    pattern = f'[{string.punctuation}]'
    return re.sub(pattern, '', text)

products = products.fillna({'review':''})
products['review_clean'] = products['review'].apply(remove_punctuation)
products = products[products['rating'] != 3]

In [ ]:
products['sentiment'] = products['rating'].apply(lambda x: 1 if x>3 else -1)

In [ ]:
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [ ]:
import json

with open('/content/drive/My Drive/ML Specialization/module-2-assignment-train-idx.json', 'r') as train_file:
    train_indices = json.load(train_file)
with open('/content/drive/My Drive/ML Specialization/module-2-assignment-test-idx.json', 'r') as test_file:
    test_indices = json.load(test_file)
    
train_data = products.iloc[train_indices]
test_data = products.iloc[test_indices]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])
# print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))
print(type(train_matrix))

121706
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
from sklearn.linear_model import LogisticRegression

sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
np.sum(sentiment_model.coef_ >= 0)

90295

In [ ]:
sample_test_data = test_data[10:13]
print(sample_test_data)

                                                 name  ... sentiment
59                          Our Baby Girl Memory Book  ...         1
71  Wall Decor Removable Decal Sticker - Colorful ...  ...        -1
91  New Style Trailing Cherry Blossom Tree Decal R...  ...        -1

[3 rows x 5 columns]


In [ ]:
print(sample_test_data.iloc[0]['review'])

Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.


In [ ]:
print(sample_test_data.iloc[1]['review'])

Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.


In [ ]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.19789313  -3.26189495 -10.94663126]


In [ ]:
scores = sentiment_model.decision_function(test_matrix)
lst = []
for i in range(len(test_data)):
    lst.append((scores[i], test_data.iloc[i]['name']))
lst.sort(reverse=True)
lst[:20]

[(50.90031166517181, 'Infantino Wrap and Tie Baby Carrier, Black Blueberries'),
 (48.143669252989994, 'Baby Einstein Around The World Discovery Center'),
 (45.53908258348218,
  'Graco FastAction Fold Jogger Click Connect Stroller, Grapeade'),
 (42.39750299310422,
  'Roan Rocco Classic Pram Stroller 2-in-1 with Bassinet and Seat Unit - Coffee'),
 (41.910421245506534, 'Diono RadianRXT Convertible Car Seat, Plum'),
 (41.143631895991895, 'Britax 2012 B-Agile Stroller, Red'),
 (40.967640997215106, 'Evenflo X Sport Plus Convenience Stroller - Christina'),
 (40.30381097048069, "Graco Pack 'n Play Element Playard - Flint"),
 (37.198039177290916, "Fisher-Price Cradle 'N Swing,  My Little Snugabunny"),
 (36.84998348542043, 'Stork Craft Beatrice Combo Tower Chest, White'),
 (36.5531726089077, 'Mamas &amp; Papas 2014 Urbo2 Stroller - Black'),
 (35.991595291943376,
  'Baby Jogger City Mini GT Double Stroller, Shadow/Orange'),
 (35.58613004726488, "P'Kolino Silly Soft Seating in Tias, Green"),
 (35.

In [ ]:
lst[-20:]

[(-20.216950382750895,
  'Motorola Digital Video Baby Monitor with Room Temperature Thermometer'),
 (-20.55817900621977, 'Regalo My Cot Portable Bed, Royal Blue'),
 (-20.735242592136725, 'Baby Jogger Summit XC Double Stroller, Red/Black'),
 (-20.921075383820135,
  'VTech Communications Safe &amp; Sound Digital Audio Monitor with two Parent Units'),
 (-21.0199439663041, 'Philips AVENT Newborn Starter Set'),
 (-21.116258836883425, 'Peg-Perego Tatamia High Chair, White Latte'),
 (-21.274255396561255, 'Safety 1st Deluxe 4-in-1 Bath Station'),
 (-21.490428082661747, 'Snuza Portable Baby Movement Monitor'),
 (-21.928440591459204, 'Ellaroo Mei Tai Baby Carrier - Hershey'),
 (-21.971725764137776, 'Safety 1st Exchangeable Tip 3 in 1 Thermometer'),
 (-22.07372535285331,
  'Samsung SEW-3037W Wireless Pan Tilt Video Baby Monitor Infrared Night Vision and Zoom, 3.5 inch'),
 (-22.295023106654007, 'Cloth Diaper Sprayer--styles may vary'),
 (-22.856335859967334, 'Baby Trend Inertia Infant Car Seat - H

In [ ]:
print(sentiment_model.score(test_matrix, test_data['sentiment']))

0.9320854331653468


In [ ]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

vectorizer_word_subset = CountVectorizer(vocabulary=significant_words)
train_matrix_word_subset = vectorizer_word_subset.transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [ ]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
np.sum(simple_model.coef_ >=0)

10

In [ ]:
print(sentiment_model.score(train_matrix, train_data['sentiment']))
print(sentiment_model.score(test_matrix, test_data['sentiment']))
print(simple_model.score(train_matrix_word_subset, train_data['sentiment']))
print(simple_model.score(test_matrix_word_subset, test_data['sentiment']))

0.9477573904179408
0.9320854331653468
0.8668225700065959
0.8693604511639069


In [ ]:
print(len(train_data))
print(len(train_data[train_data['sentiment']==1]))
print(len(test_data[test_data['sentiment']==1])/len(test_data))

133416
112164
0.8427825773938085
